In [1]:
# import itertools
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import h5py
import napari
import tifffile as tiff
import seaborn as sns
import pickle

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

df_meta_path = data_dir / 'OCT mouse' / 'Whole' / 'metadata' / 'imgs_reg.csv'
df_imgs = pd.read_csv(df_meta_path)

# Segmentation 

In [85]:
from skimage import exposure, util

def contrast_str(img, n_min=0.01, n_max=99.95):
    p2, p98 = np.percentile(img, (n_min, n_max))
    img_rescale = exposure.rescale_intensity(img, in_range=(p2, p98))
    img_rescale = util.img_as_ubyte(img_rescale)
    return img_rescale

def split_image(img, tile_row, tile_col):
    n_row = img.shape[1] // tile_row 
    n_col = img.shape[2] // tile_col 
    img_arr = []
    for i in range(0, n_row+1):  # loop through height-wise pixels of image, striding by 33
        for j in range(0, n_col+1):  # loop through width-wise pixels of image, striding by 33
            img_tile = img[:, i*tile_row : (i+1)*tile_row, j*tile_col : (j+1)*tile_col]  # creating label image from h,w pixel to 33 pixels to the right and down
            img_arr.append(img_tile)  # create list from label images
    return img_arr

def stitch_mask(masks, img, tile_row, tile_col):
    n_row = img.shape[1] // tile_row 
    n_col = img.shape[2] // tile_col 
    mask_combined = np.zeros((img.shape[1], img.shape[2])).astype(np.uint32)
    
    img_arr = []
    for i in range(0, n_row+1):  
        for j in range(0, n_col+1):  
            mask_combined[i*tile_row : (i+1)*tile_row, j*tile_col : (j+1)*tile_col] = masks[i*(n_col+1)+j]
    # mask_combined = mask_combined[:img.shape[1], :img.shape[2]]
    return mask_combined
    

In [86]:
whole_seg_path = data_dir / 'OCT mouse'  / 'Whole' / 'imgs' / 'segmentation'
mask_path = data_dir / 'OCT mouse'  / 'Whole' / 'imgs' / 'masks'

mask_path .mkdir(parents=True, exist_ok=True)

In [90]:
from cellpose import core, utils, io, models, metrics
use_GPU = core.use_gpu()
yn = ['NO', 'YES']
print(f'>>> GPU activated? {yn[use_GPU]}')


>>> GPU activated? YES


In [91]:
# declare model
model = models.CellposeModel(gpu=True, model_type='CP')

# Cyto segmentaion
masks = []
for p in os.listdir(whole_seg_path):
    img = skimage.io.imread(whole_seg_path / p).transpose((2,0,1))
    imgs = split_image(img, 2000, 2000)
    
    # run model on test images
    masks, flows, styles = model.eval(imgs, 
                                  channels=[2,3],
                                  diameter=50,
                                  flow_threshold=0.8,
                                  cellprob_threshold=-3, 
                                  )
    mask = stitch_mask(masks, img, 2000,2000)  
    
    file_path = mask_path / p
    tiff.imwrite(file_path, mask)

In [92]:
# declare model
model = models.CellposeModel(gpu=True, model_type='nuclei')

# Nuclei segmentation
masks_nuclei = []
for p in os.listdir(whole_seg_path):
    img = skimage.io.imread(whole_seg_path / p).transpose((2,0,1))
    imgs = split_image(img, 2000, 2000)
    
    # run model on test images
    masks, flows, styles = model.eval(imgs, 
                                  channels=[2,3],
                                  diameter=50,
                                  flow_threshold=0.8,
                                  cellprob_threshold=-3, 
                                  )
    mask = stitch_mask(masks, img, 2000,2000)  
    
    file_path = mask_path / f'Nuclei_{p}'
    tiff.imwrite(file_path, mask)